In [43]:
import numpy as np
import pandas as pd

In [44]:
df_train = pd.read_csv("drive/MyDrive/Dataset-Project2-CI/train_ds.csv")
df_test = pd.read_csv("drive/MyDrive/Dataset-Project2-CI/test_ds.csv")

In [45]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         2400 non-null   object 
 1   year          2069 non-null   float64
 2   price         2052 non-null   float64
 3   transmission  2400 non-null   object 
 4   mileage       2025 non-null   float64
 5   fuelType      2400 non-null   object 
 6   tax           2055 non-null   float64
 7   mpg           2064 non-null   float64
 8   engineSize    2067 non-null   float64
 9   Manufacturer  2400 non-null   object 
dtypes: float64(6), object(4)
memory usage: 187.6+ KB


In [46]:
df_train['year'] = df_train['year'].fillna(df_train['year'].mode()[0])
df_train['price'] = df_train['price'].fillna(df_train['price'].mean())
df_train['mileage'] = df_train['mileage'].fillna(df_train['mileage'].mean())
df_train['tax'] = df_train['tax'].fillna(df_train['tax'].mode()[0])
df_train['mpg'] = df_train['mpg'].fillna(df_train['mpg'].mean())
df_train['engineSize'] = df_train['engineSize'].fillna(df_train['engineSize'].mode()[0])

In [47]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         2400 non-null   object 
 1   year          2400 non-null   float64
 2   price         2400 non-null   float64
 3   transmission  2400 non-null   object 
 4   mileage       2400 non-null   float64
 5   fuelType      2400 non-null   object 
 6   tax           2400 non-null   float64
 7   mpg           2400 non-null   float64
 8   engineSize    2400 non-null   float64
 9   Manufacturer  2400 non-null   object 
dtypes: float64(6), object(4)
memory usage: 187.6+ KB


In [48]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Loop through each categorical column and encode it
for col in ['transmission', 'fuelType', 'Manufacturer','model']:
    # Fit LabelEncoder on train data
    label_encoder.fit(df_test[col])

    # Transform (encode) train and test data using the same LabelEncoder
    df_train[col + '_encoded'] = label_encoder.transform(df_train[col])
    df_test[col + '_encoded'] = label_encoder.transform(df_test[col])


label_encoder.fit(df_train['engineSize'])
df_train['engineSize' + '_encoded'] = label_encoder.transform(df_train['engineSize'])
df_test['engineSize' + '_encoded'] = label_encoder.transform(df_test['engineSize'])

In [49]:
df_train

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,Manufacturer,transmission_encoded,fuelType_encoded,Manufacturer_encoded,model_encoded,engineSize_encoded
0,Aygo,2016.0,7500.000000,Manual,14612.000000,Petrol,0.0,69.00000,1.0,toyota,1,3,6,12,1
1,Up,2018.0,8712.000000,Manual,1607.000000,Petrol,145.0,64.20000,1.0,volkswagen,1,3,8,37,1
2,Focus,2018.0,12670.000000,Manual,7162.000000,Petrol,145.0,60.10000,1.0,ford,1,3,2,19,1
3,GLA Class,2019.0,23599.000000,Semi-Auto,23931.409877,Petrol,145.0,36.20000,1.6,merc,3,3,4,20,7
4,Octavia,2019.0,18999.000000,Semi-Auto,1162.000000,Petrol,145.0,44.80000,1.5,skoda,3,3,5,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,Tiguan,2020.0,26980.000000,Manual,2194.000000,Diesel,145.0,42.20000,2.0,volkswagen,1,0,8,35,10
2396,3 Series,2017.0,16000.000000,Manual,34098.000000,Diesel,145.0,64.20000,2.0,BMW,1,0,1,2,10
2397,Fiesta,2018.0,15645.089181,Manual,23931.409877,Petrol,145.0,65.70000,2.0,ford,1,3,2,18,10
2398,Up,2017.0,7391.000000,Manual,23931.409877,Petrol,150.0,64.20000,2.0,volkswagen,1,3,8,37,10


In [50]:
df_train['engineSize'].unique()

array([1. , 1.6, 1.5, 1.4, 2. , 1.7, 1.2, 2.1, 0. , 3. , 1.3, 1.8, 4. ,
       1.1, 2.5, 3.5])

In [51]:
df_train = df_train.drop(columns=['transmission','fuelType','Manufacturer','model','engineSize'])
df_test = df_test.drop(columns=['transmission','fuelType','Manufacturer','model','engineSize'])

In [52]:
def entropy_calculate (indices):
    temp = df_train.iloc[indices]

    class_counts = np.bincount(temp['model_encoded'])

    # Compute the probability of each class
    probabilities = class_counts / len(temp['model_encoded'])

    # Ignore zero probabilities to avoid runtime warnings
    probabilities = probabilities[probabilities > 0]

    # Calculate entropy
    entropy = -np.sum(probabilities * np.log2(probabilities))

    return entropy

In [124]:
entropy_calculate(list(range(2400)))

5.018366577203956

In [53]:
def ig_calculate (column,indices,father_entropy) :
  values = column.unique()
  children = []
  for value in values :
    current_indices = df_train.index[column == value].tolist()
    entropy = entropy_calculate(current_indices)
    if (len(current_indices)!= 0):
      children.append((entropy,len(current_indices)))

  sum = 0;
  sum_weights = 0;
  for child in children :
    sum_weights += child[1]
    sum += (child[0]*child[1])

  return father_entropy - sum/sum_weights

In [54]:
best_threshold = None
best_ig = float('-inf')

# Iterate over unique values in 'price' column
for threshold in df_train['price'].unique():
    # Create a new column to represent the split based on the threshold
    df_train['price_split'] = (df_train['price'] <= threshold).astype(int)

    # Calculate the entropy of the parent node (before split)
    # father_entropy = entropy_calculate(df_train.index)

    # Calculate information gain for this split
    ig = ig_calculate(df_train['price_split'], df_train.index, 5.018366577203956)
    #print (ig)
    # Update best threshold and information gain if needed
    if ig > best_ig:
        best_threshold = threshold
        best_ig = ig

    # Remove the temporary 'price_split' column
    df_train.drop(columns=['price_split'], inplace=True)

# Print the best threshold and information gain
print("Best Threshold:", best_threshold)
print("Best Information Gain:", best_ig)

Best Threshold: 16499.0
Best Information Gain: 0.3350668988353869


In [55]:
best_threshold = None
best_ig = float('-inf')

# Iterate over unique values in 'price' column
for threshold in df_train['mileage'].unique():
    # Create a new column to represent the split based on the threshold
    df_train['mileage_split'] = (df_train['mileage'] <= threshold).astype(int)

    # Calculate the entropy of the parent node (before split)
    # father_entropy = entropy_calculate(df_train.index)

    # Calculate information gain for this split
    ig = ig_calculate(df_train['mileage_split'], df_train.index, 5.018366577203956)
    #print (ig)
    # Update best threshold and information gain if needed
    if ig > best_ig:
        best_threshold = threshold
        best_ig = ig

    # Remove the temporary 'price_split' column
    df_train.drop(columns=['mileage_split'], inplace=True)

# Print the best threshold and information gain
print("Best Threshold:", best_threshold)
print("Best Information Gain:", best_ig)

Best Threshold: 33446.0
Best Information Gain: 0.045440037625788854


In [56]:
best_threshold = None
best_ig = float('-inf')

# Iterate over unique values in 'price' column
for threshold in df_train['mpg'].unique():
    # Create a new column to represent the split based on the threshold
    df_train['mpg_split'] = (df_train['mpg'] <= threshold).astype(int)

    # Calculate the entropy of the parent node (before split)
    # father_entropy = entropy_calculate(df_train.index)

    # Calculate information gain for this split
    ig = ig_calculate(df_train['mpg_split'], df_train.index, 5.018366577203956)
    #print (ig)
    # Update best threshold and information gain if needed
    if ig > best_ig:
        best_threshold = threshold
        best_ig = ig

    # Remove the temporary 'price_split' column
    df_train.drop(columns=['mpg_split'], inplace=True)

# Print the best threshold and information gain
print("Best Threshold:", best_threshold)
print("Best Information Gain:", best_ig)

Best Threshold: 55.4
Best Information Gain: 0.1632264516047659


In [57]:
price_threshhold = 16499.0
mileage_threshhold = 33446.0
mpg_threshhold = 55.4
df_train['price_split'] = (df_train['price'] <= price_threshhold).astype(int)
df_train['mpg_split'] = (df_train['mpg'] <= mpg_threshhold).astype(int)
df_train['mileage_split'] = (df_train['mileage'] <= mileage_threshhold).astype(int)
df_test['price_split'] = (df_test['price'] <= price_threshhold).astype(int)
df_test['mpg_split'] = (df_test['mpg'] <= mpg_threshhold).astype(int)
df_test['mileage_split'] = (df_test['mileage'] <= mileage_threshhold).astype(int)

In [58]:
df_train = df_train.drop(columns=['price','mpg','mileage'])
df_test = df_test.drop(columns=['price','mpg','mileage'])

In [59]:
df_train = df_train.drop(columns=['year','tax'])
df_test = df_test.drop(columns=['year','tax'])

In [60]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   transmission_encoded  600 non-null    int64
 1   fuelType_encoded      600 non-null    int64
 2   Manufacturer_encoded  600 non-null    int64
 3   model_encoded         600 non-null    int64
 4   engineSize_encoded    600 non-null    int64
 5   price_split           600 non-null    int64
 6   mpg_split             600 non-null    int64
 7   mileage_split         600 non-null    int64
dtypes: int64(8)
memory usage: 37.6 KB


In [73]:
# Assuming df_train and df_test are your DataFrames
label_column = 'model_encoded'

# Reorder columns in df_train
df_train = df_train[[col for col in df_train.columns if col != label_column] + [label_column]]

# Reorder columns in df_test (optional if you also want to reorder the columns in the test DataFrame)
df_test = df_test[[col for col in df_test.columns if col != label_column] + [label_column]]


In [307]:
df_train['Manufacturer_encoded'].unique()

array([6, 8, 2, 4, 5, 7, 1, 0, 3])

In [315]:
class Node :
  def __init__ (self,attribute=None,value=None,depth=None,entropy=None,id=None,df=None,father_attribute = None):
    self.attribute = attribute  # Attribute Selected for this Node
    self.value = value  #value of Attribute Selected by its father
    self.depth = depth  #depth of this Node
    self.children = []  #list of its children
    self.entropy = entropy
    self.id = id
    self.checked = False
    self.df = df
    self.label = None
    self.is_leaf = False
    self.father_attribute = father_attribute

In [318]:
class DecisionTree :
  def __init__(self,max_depth = None):
    self.root = None
    self.label = None
    self.all = None
    self.nodes = []
    self.first_id_to_create = 0
    self.max_depth = max_depth if max_depth is not None else 3

  def fit (self,df):
    self.all = df
    self.label = df.columns[-1]
    root_node = Node(id=self.first_id_to_create,depth = 0,df=df)
    self.nodes.append(root_node)
    self.first_id_to_create += 1
    self.root = root_node
    self.nodes.append(root_node)
    #self.grow_tree(self.nodes[0])

    entropy = self.calculate_entropy(self.nodes[0])
    self.nodes[0].entropy = entropy
    self.grow_tree(self.nodes[0])


  def grow_tree (self,node):
    node.checked = True
    if node.depth >= self.max_depth :
      node.label = node.df[self.label].value_counts().idxmax()
      node.is_leaf = True
      return

    elif len(node.df[self.label].unique()) == 1:
      node.label = node.df[self.label].unique()[0]
      node.is_leaf = True
      return

    else :
      best_ig = -20
      best_attribute = None
      for column in node.df.columns :
        if column != 'model_encoded' and self.calculate_ig(node.df,column) > best_ig :
          best_ig = self.calculate_ig(node.df,column)
          best_attribute = column

      node.attribute = best_attribute
      for value in node.df[best_attribute].unique() :
        newNode = Node(value=value,
                       depth=(node.depth+1),
                       df=node.df[node.df[best_attribute]==value].drop(columns=[best_attribute]) ,
                       id=(self.first_id_to_create)+1,
                       father_attribute = node.attribute);
        self.first_id_to_create += 1
        self.nodes.append(newNode)
        self.grow_tree(newNode)
        node.children.append(newNode)




  def calculate_entropy(self,node):
    unique_labels, label_counts = np.unique(node.df[self.label], return_counts=True)
    probabilities = label_counts / len(node.df[self.label])
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

  def calculate_entropy2(self,df):
    unique_labels, label_counts = np.unique(df[self.label], return_counts=True)
    probabilities = label_counts / len(df[self.label])
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

  def calculate_ig (self,df,attr):
    entropy_father = self.calculate_entropy2(df)
    entropy_children = 0
    sum_weights = 0
    for value in df[attr].unique():
      weight = len(df[df[attr] == value])
      entropy_child = self.calculate_entropy2(df[df[attr]==value])
      entropy_children += (weight*entropy_child)
      sum_weights += weight

    return entropy_father - (entropy_children/sum_weights)

  def predict(self, df_test):
        predictions = []
        for _, row in df_test.iterrows():
            node = self.root
            while not node.is_leaf:
                attribute_value = row[node.attribute]
                found_child = False
                for child in node.children:
                    if child.value == attribute_value:
                        node = child
                        found_child = True
                        break
                if not found_child:
                    # If the attribute value is not found in the children,
                    # we cannot make a prediction. This might happen if the
                    # test data contains a value not seen during training.
                    predictions.append(None)
                    break
            else:
                predictions.append(node.label)
        return predictions


In [335]:
dt = DecisionTree(max_depth=4)
dt.fit(df_train)

predictions = dt.predict(df_test)

actual_labels = df_test['model_encoded']

accuracy_score = 0
for i in range(600) :
  if predictions[i] == actual_labels[i] :
    accuracy_score+=1

print(accuracy_score/600)

0.65
